In [ ]:
# Import the drive module from the google.colab package
from google.colab import drive

# Mount Google Drive to the Colab environment, which allows access to files stored in my Google Drive
drive.mount('/content/drive')


In [ ]:
# Use the magic command "%pip" to install the "ultralytics" package. This command installs packages directly from the Python Package Index (PyPI).
%pip install ultralytics

# Use the "!pip" command to install the "ultralytics" package. This command installs packages directly from the Python Package Index (PyPI). This is the same command as before but sometimes this command line does not work
!pip install ultralytics

# Use the "!pip" command to install the "Squarify" package, which is used for creating treemaps in Python.
!pip install Squarify


In [5]:
# Import the YOLO class from the ultralytics package, which provides functionality for working with YOLO object detection models.
from ultralytics import YOLO

# Import the squarify module, which is used for creating treemaps in Python.
import squarify

# Import the pyplot module from the matplotlib library, which is used for creating visualizations in Python.
import matplotlib.pyplot as plt

# Import the cv2 module, which is the OpenCV library for computer vision tasks.
import cv2

# Import the os module, which provides a portable way of using operating system-dependent functionality in Python.
import os

# Import the random module, which provides functions for generating random numbers and performing random selections.
import random

# Import the pandas library, which provides data structures and data analysis tools for Python.
import pandas as pd

# Import the image module from the matplotlib library, which is used for working with images in Python.
import matplotlib.image as mpimg

# Import the seaborn library, which is used for statistical data visualization in Python.
import seaborn as sns

# Set the seaborn plotting style to 'darkgrid' for visual consistency.
sns.set_style('darkgrid')

# Use the "%matplotlib inline" magic command to enable inline plotting in Jupyter notebooks, which displays plots directly within the notebook.
%matplotlib inline


In [ ]:
# Define the paths to the directories containing training, testing, and validation images and labels
train_images = "/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/train/images"
train_labels = "/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/train/labels"

test_images = "/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/test/images"
test_labels = "/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/test/labels"

val_images = "/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/valid/images"
val_labels = "/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/valid/labels"

# Get a list of all the image files in the training images directory (which is in the mounted google drive)
image_files = os.listdir(train_images)

# Choose 16 random image files from the list
random_images = random.sample(image_files, 16)

# Set up the plot with 4 rows and 4 columns, each subplot represents an image with object detections
fig, axs = plt.subplots(4, 4, figsize=(16, 16))

# Loop over the randomly selected image files
for i, image_file in enumerate(random_images):
    row = i // 4  # Calculate the row index for the subplot
    col = i % 4   # Calculate the column index for the subplot

    # Load the image
    image_path = os.path.join(train_images, image_file)
    image = cv2.imread(image_path)

    # Load the labels for this image
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")

    # Loop over the labels and plot the object detections on the image
    for label in labels:
        if len(label.split()) != 5:
            continue
        class_id, x_center, y_center, width, height = map(float, label.split())
        x_min = int((x_center - width/2) * image.shape[1])
        y_min = int((y_center - height/2) * image.shape[0])
        x_max = int((x_center + width/2) * image.shape[1])
        y_max = int((y_center + height/2) * image.shape[0])
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)

    # Show the image with the object detections on the corresponding subplot
    axs[row, col].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[row, col].axis('off')  # Hide axis

# Display the plot
plt.show()


In [ ]:
# Load an image using OpenCV imread() function. Provide the path to the image file as an argument.
image = cv2.imread("/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/test/images/02e39612d_jpg.rf.cc5483bb711f080d12b644ff62cf977a.jpg")

# Get the size of the image using the shape attribute of the image array.
# The shape attribute returns a tuple containing height, width, and number of channels (if it's a color image).
height, width, channels = image.shape

# Print the dimensions and channels of the image using formatted string.
print(f"The image has dimensions {width}x{height} and {channels} channels.")


In [ ]:
# Load a pre-trained YOLOv8 model named 'yolov8x.pt' using the YOLO class from the ultralytics package.
model = YOLO('yolov8x.pt')

# Train the loaded model using the train() method.
# Provide the path to the data.yaml file which contains information about the dataset.
# Set the number of epochs to train for to 50.
# Set the input image size (imgsz) to the height of the previously loaded image.
# Set the random seed (seed) for reproducibility to 42.
# Set the batch size (batch) to 8 images per batch.
# Set the number of workers (workers) to 4 for data loading.
model.train(data='/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/data.yaml',
            epochs=50,
            imgsz=height,
            seed=42,
            batch=8,
            workers=4)


In [ ]:
# Enable inline plotting for matplotlib in Jupyter notebooks
%matplotlib inline

# Read the results.csv file as a pandas DataFrame
df = pd.read_csv('/content/runs/detect/train/results.csv')

# Strip leading and trailing whitespaces from column names
df.columns = df.columns.str.strip()

# Create subplots grid with 5 rows and 2 columns using seaborn
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

# Plot each column using seaborn's lineplot function and assign each plot to a specific subplot
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])

# Set titles and axis labels for each subplot
axs[0,0].set(title='Train Box Loss')
axs[0,1].set(title='Train Class Loss')
axs[1,0].set(title='Train DFL Loss')
axs[1,1].set(title='Metrics Precision (B)')
axs[2,0].set(title='Metrics Recall (B)')
axs[2,1].set(title='Metrics mAP50 (B)')
axs[3,0].set(title='Metrics mAP50-95 (B)')
axs[3,1].set(title='Validation Box Loss')
axs[4,0].set(title='Validation Class Loss')
axs[4,1].set(title='Validation DFL Loss')

# Add suptitle for the entire figure
plt.suptitle('Training Metrics and Loss', fontsize=24)

# Adjust top margin to make space for suptitle
plt.subplots_adjust(top=0.8)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Enable inline plotting for matplotlib in Jupyter notebooks
%matplotlib inline

# Load the best performing model from the specified google drive path
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Evaluate the model on the test dataset
# Set the confidence threshold to 0.25 and specify that evaluation should be performed on the test split
metrics = model.val(conf=0.25, split='test')


In [ ]:
# Enable inline plotting for matplotlib in Jupyter notebooks
%matplotlib inline

# Create a barplot using seaborn's barplot function
# x specifies the labels for the x-axis (metrics) and y specifies the values for the y-axis (mAP values)
ax = sns.barplot(x=['mAP50-95', 'mAP50', 'mAP75'], y=[metrics.box.map, metrics.box.map50, metrics.box.map75])

# Set the title and axis labels
ax.set_title('YOLO Evaluation Metrics')
ax.set_xlabel('Metric')
ax.set_ylabel('Value')

# Set the figure size
fig = plt.gcf()
fig.set_size_inches(8, 6)

# Add the values on top of the bars
for p in ax.patches:
    ax.annotate('{:.3f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')

# Show the plot
plt.show()


In [ ]:
# Enable inline plotting for matplotlib in Jupyter notebooks
%matplotlib inline

# Read the confusion matrix image file in the mounted google drive path using matplotlib's imread function
img = mpimg.imread('/content/runs/detect/train/confusion_matrix.png')

# Plot the confusion matrix image
# Create a figure and axis object with a specified figsize
fig, ax = plt.subplots(figsize=(15, 15))

# Display the image on the axis
ax.imshow(img)

# Turn off the axis to remove axis ticks and labels
ax.axis('off')

# Show the plot
plt.show()


In [13]:
# Define a function named ship_detect that takes the path to an image as input
def ship_detect(img_path):

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Pass the image through the detection model and get the result
    detect_result = model(img)

    # Plot the detections on the image
    detect_img = detect_result[0].plot()

    # Convert the image from BGR (OpenCV format) to RGB format (matplotlib format)
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    # Return the image with ship detections
    return detect_img


In [ ]:
# Import the random module
import random

# Define the directory where the custom images are stored (which is the mounted google drive, all images and data is stored in that)
custom_image_dir = '/content/drive/MyDrive/Dissertation/Experiments/Dataset/ships-aerial-images/test/images'

# Get the list of image files in the directory
image_files = os.listdir(custom_image_dir)

# Select 16 random images from the list
selected_images = random.sample(image_files, 16)

# Create a figure with subplots for each image
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

# Iterate over the selected images and plot each one
for i, img_file in enumerate(selected_images):

    # Compute the row and column index of the current subplot
    row_idx = i // 4
    col_idx = i % 4

    # Load the current image and run object detection using the ship_detect function
    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = ship_detect(img_path)

    # Plot the current image on the appropriate subplot
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)
